In [6]:
# -*- coding: utf-8 -*-
"""
Created on Sun Apr 22 18:20:07 2018

@author: abinaya
"""

import pandas as pd
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# 檢查導入是否成功
print("Starting imports...")
print("pandas imported:", pd.__version__)
print("sklearn imported:", sklearn.__version__)
print("train_test_split imported:", train_test_split)

def preprocessing_features(df_train, df_test, process_continuous):
    to_delete_features = ['default','pdays']
    continuous_features = ['age', 'campaign', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']
    categorical_ordered_features = ['education', 'housing', 'loan', 'contact','month', 'day_of_week','poutcome']
    categorical_unordered_features = ['job', 'marital']
    
    unknown_present_features = ['job','marital','education','housing','loan']
    
    ### Delete Features
    for feat in to_delete_features:
        print(f"\n--------- deleting feature --------- {feat}") 
        del df_train[feat]
        del df_test[feat]
    
    ### Fill unknowns in the features with feature-mode
    for feat in unknown_present_features:
        print(f"\n--------- Replacing unknowns in feature --------- {feat}") 
        feature_value_counts = df_train[feat].value_counts()
        print(f"Replaced with: {feature_value_counts.idxmax()}")
        df_train.loc[df_train[feat] == "unknown",feat] = feature_value_counts.idxmax()
        df_test.loc[df_test[feat] == "unknown",feat] = feature_value_counts.idxmax()
    
    
    ### Label Categorical Ordered Features
    label_dict = {'education':{'illiterate':0, 'basic.4y':4, 'basic.6y':6, 'basic.9y':9, 'high.school':11, 'professional.course':13, 'university.degree':14},
                  'housing':{'no':0,'yes':1},
                  'loan':{'no':0,'yes':1},
                  'contact':{'telephone':0,'cellular':1},
                  'month':{'jan':1,'feb':2,'mar':3,'apr':4,'may':5,'jun':6,'jul':7,'aug':8,'sep':9,'oct':10,'nov':11,'dec':12},
                  'day_of_week':{'mon':1,'tue':2,'wed':3,'thu':4,'fri':5,'sat':6,'sun':7},
                  'poutcome':{'nonexistent':0,'failure':1,'success':2}}
    
    for feat in categorical_ordered_features:
        print(f"\n--------- Labelling feature --------- {feat}") 
        df_train = df_train.replace({feat:label_dict[feat]})
        df_test = df_test.replace({feat:label_dict[feat]})
        print(f"Labelled as: {label_dict[feat]}") 

        
    ### One hot encoding Categorical Un-ordered Features  
    for feat in categorical_unordered_features:
        print(f"\n--------- One Hot Encoding feature --------- {feat}")
        label_encoder = LabelEncoder()
        label_encoder.fit(df_train[feat])
        df_train[feat] = label_encoder.transform(df_train[feat])
        df_test[feat] = label_encoder.transform(df_test[feat])

    one_hot_encoder = OneHotEncoder(sparse_output=False)
    one_hot_encoder.fit(df_train[categorical_unordered_features])
    one_hot_encoded_array_train = one_hot_encoder.transform(df_train[categorical_unordered_features])
    one_hot_encoded_df_train = pd.DataFrame(one_hot_encoded_array_train, index=df_train.index)
    one_hot_encoded_array_test = one_hot_encoder.transform(df_test[categorical_unordered_features])
    one_hot_encoded_df_test = pd.DataFrame(one_hot_encoded_array_test, index=df_test.index)
        
    df_train = pd.concat([df_train,one_hot_encoded_df_train], axis=1) #concatenate old columns with new one hot encoded columns
    df_test = pd.concat([df_test,one_hot_encoded_df_test], axis=1) #concatenate old columns with new one hot encoded columns

    df_train = df_train.drop(categorical_unordered_features, axis=1) #Delete columns which were one hot encoded
    df_test = df_test.drop(categorical_unordered_features, axis=1) #Delete columns which were one hot encoded

    ### Normalization or Standardization of Continuous Features
    if process_continuous == "Standardize":
        print("\n--------- Standardizing Continuous Features (Mean=0, Standard Deviation=1) --------- ")  # 更新為 Python 3 語法
        standardization = StandardScaler()
        standardization.fit(df_train[continuous_features])
        df_train[continuous_features] = standardization.transform(df_train[continuous_features])
        df_test[continuous_features] = standardization.transform(df_test[continuous_features])

    elif process_continuous == "Normalize":
        print("\n--------- Normalizing Continuous Features (Min=0, Max=1) --------- ")  # 更新為 Python 3 語法
        min_max_scaling = MinMaxScaler()
        min_max_scaling.fit(df_train[continuous_features])
        df_train[continuous_features] = min_max_scaling.transform(df_train[continuous_features])
        df_test[continuous_features] = min_max_scaling.transform(df_test[continuous_features])

    ### Return pre-processed df
    return df_train, df_test

def preprocessing_class(df_train, df_test):
    print("\n--------- Labelling Class Information --------- ")  # 更新為 Python 3 語法
    class_col = 'y'
    label_dict = {class_col:{'no':0,'yes':1}}
    df_train = df_train.replace({class_col:label_dict[class_col]})
    df_test = df_test.replace({class_col:label_dict[class_col]})
    return df_train, df_test
    

df = pd.read_csv('Data/bank-additional.csv')

# 我們將數據分為訓練集和測試集
df_train, df_test = train_test_split(df, test_size=0.25, random_state=42, stratify=df['y'])


# 預處理類別標籤
df_train_class = pd.DataFrame(df_train['y'])
df_test_class = pd.DataFrame(df_test['y'])
df_train_class, df_test_class = preprocessing_class(df_train_class, df_test_class)

# 預處理特徵
df_train_features = df_train.loc[:, df_train.columns != 'y']
df_test_features = df_test.loc[:, df_test.columns != 'y']
df_train_features, df_test_features = preprocessing_features(df_train_features, df_test_features, "Normalize")

# df_class = pd.DataFrame(df['y'])
# df_class = preprocessing_class(df_class)

# df_features = df.loc[:, df.columns != 'y']
# df_features = preprocessing_features(df_features,"Normalize")

# 合併特徵和類別
df_train_preprocessed = pd.concat([df_train_features, df_train_class], axis=1)
df_test_preprocessed = pd.concat([df_test_features, df_test_class], axis=1)

# df_preprocessed = pd.concat([df_features, df_class], axis=1)
# df_preprocessed.to_csv('bank-additional-preprocessed-mode-normalize.csv')

# 保存預處理後的數據
df_train_preprocessed.to_csv('Data/bank-additional-preprocessed-mode-normalize-train.csv', index=False)
df_test_preprocessed.to_csv('Data/bank-additional-preprocessed-mode-normalize-test.csv', index=False)

Starting imports...
pandas imported: 2.2.3
sklearn imported: 1.6.1
train_test_split imported: <function train_test_split at 0x158d1db20>

--------- Labelling Class Information --------- 

--------- deleting feature --------- default

--------- deleting feature --------- pdays

--------- Replacing unknowns in feature --------- job
Replaced with: admin.

--------- Replacing unknowns in feature --------- marital
Replaced with: married

--------- Replacing unknowns in feature --------- education
Replaced with: university.degree

--------- Replacing unknowns in feature --------- housing
Replaced with: yes

--------- Replacing unknowns in feature --------- loan
Replaced with: no

--------- Labelling feature --------- education
Labelled as: {'illiterate': 0, 'basic.4y': 4, 'basic.6y': 6, 'basic.9y': 9, 'high.school': 11, 'professional.course': 13, 'university.degree': 14}

--------- Labelling feature --------- housing
Labelled as: {'no': 0, 'yes': 1}

--------- Labelling feature --------- loa

/var/folders/qz/rm2scxlx4hx9h4r2tsxl0c8w0000gn/T/ipykernel_9303/4156783784.py:104: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_train = df_train.replace({class_col:label_dict[class_col]})
/var/folders/qz/rm2scxlx4hx9h4r2tsxl0c8w0000gn/T/ipykernel_9303/4156783784.py:105: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_test = df_test.replace({class_col:label_dict[class_col]})
/var/folders/qz/rm2scxlx4hx9h4r2tsxl0c8w0000gn/T/ipykernel_9303/4156783784.py:56: FutureWarning: Downcasting behavior in `replace` is deprecated and will 